In [1]:
import yfinance as yf
from pandas import DataFrame
import pandas as pd
from datetime import date
import openai
from typing import Dict, Any
import os
from tqdm import tqdm
from tr4der.utils.data_loader import DataLoader
from tr4der.utils.gpt_helper import GptHelper
from tr4der.tr4der import Tr4der

inst = Tr4der("I want to pair trade uber and lyft over the past 5 years and be short lyft and long uber.")

Loading data...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 2/2 [00:00<00:00,  3.86it/s]


                 UBER       LYFT
Date                            
2019-03-29        NaN  78.290001
2019-04-01        NaN  69.010002
2019-04-02        NaN  68.970001
2019-04-03        NaN  70.000000
2019-04-04        NaN  72.000000
...               ...        ...
2023-09-25  44.910000   9.780000
2023-09-26  44.270000   9.920000
2023-09-27  45.139999  10.320000
2023-09-28  46.139999  10.510000
2023-09-29  45.990002  10.540000

[1135 rows x 2 columns]
[{'role': 'system', 'content': 'You are a quantitative analyst. You are given a prompt. You need to identify the type of strategy that the user is looking for. Return only the strategy identifier, no markdown or comments.'}, {'role': 'user', 'content': "Return the strategy that matches that of the prompt with the correct arguments and parameters in place. If there is no match, return 'other'.  Here are the strategies: ['long_only', 'pair_trade', 'short_only', 'other'].  Here is the prompt: [I want to pair trade uber and lyft over the past 5

In [1]:
from tr4der.trades import SimpleTrades


[method for method in dir(SimpleTrades) if callable(getattr(SimpleTrades, method)) and not (method.startswith('__') and method.endswith('__'))] + ['other']



['long_only', 'pair_trade', 'short_only', 'other']

In [2]:
getattr(SimpleTrades, 'pair_trade')

<function tr4der.trades.SimpleTrades.pair_trade(self, df: pandas.core.frame.DataFrame, long_ticker: str, short_ticker: str) -> None>